In [1]:
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import string
from math import comb
import itertools
import pysam
from pyranges import PyRanges
import pairtools
import pairtools.lib.headerops as phead

# Load align table (concatemers)

In [2]:
fpath = '/scratch/indikar_root/indikar1/shared_data/single_cell/align_table/o1b01.GRCm39.align_table.parquet'
align = pd.read_parquet(fpath)
align['order'] = align.groupby('read_name')['align_id'].transform('nunique')
align = align[align['order'] > 3]
# align.head()

In [3]:
rand_read = np.random.choice(align['read_name'].unique(), 1)[0]
# rand_read = "582b1316-08d9-4ce9-8087-029913011757"
# rand_read = "e7fdbcdc-71be-4049-9050-3747b0023bf5"
print(f"{rand_read=}")

concatemer = align[align['read_name'] == rand_read]
print(f"{len(concatemer)=}")
concatemer[['read_name', 
      'align_id',
      'read_start', 
      'read_end', 
      'length_on_read', 
      'chrom',
      'fragment_id',
      'order',
      ]]

rand_read='cf49a76d-fc95-4a47-9d70-d39349c5b39e_1'
len(concatemer)=7


,read_name,align_id,read_start,read_end,length_on_read,chrom,fragment_id,order
952,cf49a76d-fc95-4a47-9d70-d39349c5b39e_1,1009,0,77,77,13,8992727.0,7
953,cf49a76d-fc95-4a47-9d70-d39349c5b39e_1,1010,77,149,72,14,9497674.0,7
954,cf49a76d-fc95-4a47-9d70-d39349c5b39e_1,1011,149,496,347,5,3407288.0,7
955,cf49a76d-fc95-4a47-9d70-d39349c5b39e_1,1012,496,718,222,13,8992730.0,7
956,cf49a76d-fc95-4a47-9d70-d39349c5b39e_1,1013,718,767,49,NaN,NaN,7
957,cf49a76d-fc95-4a47-9d70-d39349c5b39e_1,1014,767,843,76,NaN,NaN,7
958,cf49a76d-fc95-4a47-9d70-d39349c5b39e_1,1015,843,1230,387,13,8992735.0,7


In [4]:
# def plot_aligned_reads(df):
#     fig, ax = plt.subplots(figsize=(10, 2))
#     y_pos = 0.5
#     rect_height = 0.8

#     # Create a colormap based on unique chromosomes
#     unique_chroms = df['chrom'].unique()
#     cmap = plt.get_cmap('tab20b')  # You can choose other colormaps
#     norm = mcolors.Normalize(vmin=0, vmax=len(unique_chroms) - 1)

#     print("Alignment Information:")
#     print("---------------------")
#     for i, row in df.iterrows():
#         start = row['read_start']
#         length = row['length_on_read']
#         chrom = row['chrom']

#         # Color based on chromosome
#         chrom_index = list(unique_chroms).index(chrom)
#         color = cmap(norm(chrom_index))

#         # Label using the chromosome value
#         label = chrom

#         # Rectangle
#         rect = patches.Rectangle((start, y_pos), length, rect_height, linewidth=1,
#                                 edgecolor='k', facecolor=color)
#         ax.add_patch(rect)

#         # Label (centered above, larger font, background)
#         ax.text(start + length/2, y_pos + rect_height/2 - 0.05, label,
#                 ha='center', va='bottom', weight='bold', fontsize=10,
#                 bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

#         # Improved Print Statement
#         print(f"chr {label}:  "
#               f"(mq: {row['mapping_quality']})")

#     # Formatting (same as before)
#     ax.set_xlabel('Read Position')
#     ax.set_title(df['read_name'].unique()[0])
#     ax.set_yticks([])
#     ax.margins(y=0.35)

#     # Improved plot limits (same as before)
#     min_start = df['read_start'].min()
#     max_end = (df['read_start'] + df['length_on_read']).max()
#     x_padding = (max_end - min_start) * 0.05
#     ax.set_xlim(min_start - x_padding, max_end + x_padding)
#     plt.show()
    
# plt.rcParams['figure.dpi'] = 200
# plot_aligned_reads(concatemer)

In [5]:
def print_concatemer(df, read=True, direct=True, expanded=True):
    """Prints consecutive pairs of chromosomes based on order.

    Args:d
        df: A pandas DataFrame with columns 'chrom' and 'order'.
    """
    chroms = df['chrom'].astype(str).to_list()

    num_chroms = len(df)
    num_pairs = num_chroms - 1
    num_expanded_pairs = comb(num_chroms, 2)

    # Concatemer (using f-string formatting)
    if read:
        print(f"\nConcatemer ({num_chroms}-way):")
        print(" --- ".join(chroms))

    # Direct Pairs (using list comprehension for a compact loop)
    if direct:
        print(f"\nDirect Pairs ({num_pairs}):")
        [print(f"{c1} --- {c2}") for c1, c2 in zip(chroms, chroms[1:])]

    # Expanded3 Pairs (iterating over combinations, similar to direct pairs)
    if expanded:
        print(f"\nExpanded Pairs ({num_expanded_pairs}):")
        [print(f"{c1} --- {c2}") for c1, c2 in itertools.combinations(chroms, 2)]
    
print_concatemer(concatemer)


Concatemer (7-way):
13 --- 14 --- 5 --- 13 --- nan --- nan --- 13

Direct Pairs (6):
13 --- 14
14 --- 5
5 --- 13
13 --- nan
nan --- nan
nan --- 13

Expanded Pairs (21):
13 --- 14
13 --- 5
13 --- 13
13 --- nan
13 --- nan
13 --- 13
14 --- 5
14 --- 13
14 --- nan
14 --- nan
14 --- 13
5 --- 13
5 --- nan
5 --- nan
5 --- 13
13 --- nan
13 --- nan
13 --- 13
nan --- nan
nan --- 13
nan --- 13


# Load pairs

In [6]:
def load_pairs(fpath, comment_char="#"):
    """Loads a pairs table from a tab-separated file.

    Args:
        fpath (str): The file path to the pairs table.
        comment_char (str, optional): The character indicating comment lines. 
                                     Defaults to "#".

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the pairs table data.
    """
    header = phead.get_header(open(fpath))[0][-1]
    header = header.replace("#columns: ", "")
    header = header.split(" ")
    
    df = pd.read_csv(fpath, 
                     sep='\t', 
                     header=None, 
                     names=header, 
                     comment="#")
    return df


direct_fpath = '/scratch/indikar_root/indikar1/shared_data/single_cell/pairs/o1b01.GRCm39.direct.pairs'
expanded_fpath = '/scratch/indikar_root/indikar1/shared_data/single_cell/pairs/o1b01.GRCm39.expanded.pairs'

direct_pairs = load_pairs(direct_fpath)
print(f"full table: {direct_pairs.shape=}")

expanded_pairs = load_pairs(expanded_fpath)
print(f"full table: {expanded_pairs.shape=}")

direct_pairs = direct_pairs[direct_pairs['readID'] == rand_read]
expanded_pairs = expanded_pairs[expanded_pairs['readID'] == rand_read]
print()
print(f"read pairs: {direct_pairs.shape=}")
print(f"read pairs: {expanded_pairs.shape=}")

full table: direct_pairs.shape=(1035, 26)
full table: expanded_pairs.shape=(4030, 26)

read pairs: direct_pairs.shape=(6, 26)
read pairs: expanded_pairs.shape=(21, 26)


In [7]:
def f(string):
    return string.replace('!', 'nan')
    

print(f"Direct Pairs from File: ({len(direct_pairs)})")
for _, row in direct_pairs.iterrows():
    print(f(row['chrom1']),  " --- ", f(row['chrom2']))

print()    
print_concatemer(concatemer, read=True, expanded=False,)

Direct Pairs from File: (6)
13  ---  14
14  ---  5
5  ---  13
13  ---  nan
nan  ---  nan
nan  ---  13


Concatemer (7-way):
13 --- 14 --- 5 --- 13 --- nan --- nan --- 13

Direct Pairs (6):
13 --- 14
14 --- 5
5 --- 13
13 --- nan
nan --- nan
nan --- 13


In [8]:
direct_pairs[[ 'chrom1',  'chrom2', 'pair_type', 'walk_pair_index', 'walk_pair_type']]

,chrom1,chrom2,pair_type,walk_pair_index,walk_pair_type
880,13,14,UU,1,R1
881,14,5,UU,2,R1
882,5,13,UU,3,R1
883,13,!,UN,4,R1
884,!,!,NN,5,R1
885,!,13,NU,6,R1


In [9]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [12]:
print(f"Expanded Pairs from File: ({len(expanded_pairs)})")
for _, row in expanded_pairs.iterrows():
    print(f(row['chrom1']),  " --- ", f(row['chrom2']))

print()    
print_concatemer(concatemer, read=True, direct=False, expanded=True,)

Expanded Pairs from File: (21)
13  ---  5
13  ---  13
13  ---  nan
13  ---  nan
13  ---  13
13  ---  14
14  ---  13
14  ---  nan
14  ---  nan
14  ---  13
14  ---  14
5  ---  nan
5  ---  nan
5  ---  13
5  ---  14
13  ---  nan
13  ---  13
13  ---  14
nan  ---  13
nan  ---  14
nan  ---  14


Concatemer (7-way):
13 --- 14 --- 5 --- 13 --- nan --- nan --- 13

Expanded Pairs (21):
13 --- 14
13 --- 5
13 --- 13
13 --- nan
13 --- nan
13 --- 13
14 --- 5
14 --- 13
14 --- nan
14 --- nan
14 --- 13
5 --- 13
5 --- nan
5 --- nan
5 --- 13
13 --- nan
13 --- nan
13 --- 13
nan --- nan
nan --- 13
nan --- 13


In [ ]:
break

In [ ]:
break

In [ ]:
break

In [ ]:
clique = []

for c1 in concatemer['chrom'].values:
    for c2 in concatemer['chrom'].values:
        clique.append((c1, c2))
        
print(len(clique))

In [ ]:
break

In [ ]:
clique = list(itertools.combinations(concatemer['chrom'].values, 2))
print(len(clique))

In [ ]:
clique

In [ ]:
break

In [ ]:
print(pairs['readID'].nunique())

In [ ]:
print(align['read_name'].nunique())

# load alignments

In [ ]:
fpath = '/scratch/indikar_root/indikar1/shared_data/single_cell/annotate/o1b01.GRCm39.pe.bam'

read_names = []

for read in pysam.AlignmentFile(fpath):
    if read.qname.split(":")[0] == rand_read:
        print(read.reference_name)
        
    read_names.append(read.qname.split(":")[0])
    
print()
print("n total reads:", len(set(read_names)))

In [ ]:
read.qname

In [ ]:
break

In [ ]:
# check test
fpath = '/scratch/indikar_root/indikar1/shared_data/single_cell/test/o1b01.test.pairs'
pairs = load_pairs(fpath)
# print(f"{pairs.shape=}")
print(len(pairs[pairs['readID'] == rand_read]))
pairs[pairs['readID'] == rand_read]

In [ ]:
break